In [2]:
%pip install pandas
import pandas as pd
import ast


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv('./dataset/tripadvisor-barcelona-restaurants-100reviews-2025-02-13.csv')

cuisines = df['cuisines'].dropna().apply(lambda x: [cuisine.strip() for cuisine in x.split(',')]).explode()

unique_cuisines = cuisines.unique()

print(f"Valores únicos en 'cuisine': {len(unique_cuisines)} valores diferentes")
print(unique_cuisines)

price_level = df['price_level'].unique()
print(f"Valores únicos en 'price_level': {len(price_level)} valores diferentes")
print(price_level)


Valores únicos en 'cuisine': 102 valores diferentes
['Steakhouse' 'Mediterranean' 'Barbecue' 'Spanish' 'Seafood' 'European'
 'Catalan' 'Cafe' 'Contemporary' 'Mexican' 'Latin' 'South American'
 'Central American' 'Fast food' 'Healthy' 'Italian' 'Pizza' 'Grill' 'Bar'
 'Pub' 'Japanese' 'Sushi' 'Asian' 'Deli' 'Gastropub' 'American' 'French'
 'International' 'Fusion' 'Diner' 'Argentinian' 'Japanese Fusion'
 'Wine Bar' 'Neapolitan' 'Campania' 'Southern-Italian' 'Brew Pub' 'Soups'
 'Chinese' 'Lebanese' 'Middle Eastern' 'Romana' 'Lazio' 'Central-Italian'
 'Central Asian' 'African' 'Ethiopian' 'German' 'Central European'
 'Polish' 'Chilean' 'Street Food' 'British' 'Indian' 'Peruvian' 'Thai'
 'Caucasian' 'Greek' 'Hawaiian' 'Balti' 'Sardinian' 'Russian' 'Caribbean'
 'Venezuelan' 'Portuguese' 'Taiwanese' 'Yunnan' 'Sicilian' 'Brazilian'
 'Azerbaijani' 'Korean' 'Vietnamese' 'Indonesian' 'Moroccan' 'Philippine'
 'Singaporean' 'Dining bars' 'Israeli' 'Tuscan' 'Belgian' 'Pakistani'
 'Persian' 'Southwes

In [4]:
valid_cuisines = [
    'Spanish', 'French', 'Italian', 'Mexican', 'Latin', 'South American',
    'Central American', 'Italian', 'American', 'Japanese', 'Chinese', 
    'Lebanese', 'Middle Eastern', 'Argentinian', 'Japanese Fusion', 
    'Neapolitan', 'Campania', 'Southern-Italian', 'Chinese', 'Lebanese', 
    'Polish', 'Chilean', 'British', 'Indian', 'Peruvian', 'Thai', 'Greek', 
    'Hawaiian', 'Sardinian', 'Russian', 'Caribbean', 'Venezuelan', 'Portuguese', 
    'Taiwanese', 'Brazilian', 'Azerbaijani', 'Korean', 'Vietnamese', 'Indonesian', 
    'Moroccan', 'Philippine', 'Israeli', 'Pakistani', 'Persian', 'Turkish', 
    'Colombian', 'Cuban', 'Georgian', 'Ecuadorean', 'Native American'
]

def extract_last_location(loc_str):
    try:
        loc_list = ast.literal_eval(loc_str)
        return loc_list[-1] if loc_list else None
    except (ValueError, SyntaxError):
        return None

df['location'] = df['original_location'].apply(extract_last_location)

df['cuisine_nation'] = df['cuisines'].apply(lambda x: [cuisine.strip() for cuisine in x.split(',')] if isinstance(x, str) else None)
df['cuisine_nation'] = df['cuisine_nation'].apply(lambda x: next((c for c in x if c in valid_cuisines), None) if x else None)

def extract_certificate_of_excellence(award_str):
    if isinstance(award_str, str):
        index = award_str.find('Certificate of Excellence')
        if index != -1:  # Si encontramos 'Certificate of Excellence'
            return award_str[index:index+30]  # 'Certificate of Excellence' tiene 26 caracteres
    return None

df['last_certificate_excellence'] = df['awards'].apply(lambda x: extract_certificate_of_excellence(x) if isinstance(x, str) else None)

df['top_tag'] = df['top_tags'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else None)

columns_to_keep = [
    'price_level', 'vegan_options', 'gluten_free', 
    'open_days_per_week', 'avg_rating', 'total_reviews_count', 
    'food', 'service', 'atmosphere', 'excellent', 'location', 
    'last_certificate_excellence', 'cuisine_nation', 'meals'
]

df_filtered = df[columns_to_keep]

df_filtered.to_csv('./dataset/columnrefactor.csv', index=False)


In [7]:
cuisine_unique_values = df['cuisine_nation'].unique()
award_unique_values = df['last_certificate_excellence'].unique()
meals_unique_values = df['meals'].unique()

print(f"Valores únicos en 'cuisine': {len(cuisine_unique_values)} valores diferentes")
print(cuisine_unique_values)

print(f"\nValores únicos en 'last_certificate_excellence': {len(award_unique_values)} valores diferentes")
print(award_unique_values)

print(f"\nValores únicos en 'meals': {len(meals_unique_values)} valores diferentes")
print(meals_unique_values)


Valores únicos en 'cuisine': 39 valores diferentes
['Spanish' None 'Mexican' 'Italian' 'Japanese' 'American' 'French' 'Latin'
 'Japanese Fusion' 'Argentinian' 'Chinese' 'Lebanese' 'Polish' 'Chilean'
 'British' 'Indian' 'Peruvian' 'Thai' 'Greek' 'Hawaiian' 'Russian'
 'Caribbean' 'Portuguese' 'Brazilian' 'Azerbaijani' 'Philippine' 'Korean'
 'Vietnamese' 'Middle Eastern' 'Venezuelan' 'Southern-Italian' 'Pakistani'
 'Indonesian' 'Moroccan' 'Turkish' 'Colombian' 'Central American'
 'South American' 'Cuban']

Valores únicos en 'last_certificate_excellence': 11 valores diferentes
['Certificate of Excellence 2016' None 'Certificate of Excellence 2020'
 'Certificate of Excellence 2019' 'Certificate of Excellence 2018'
 'Certificate of Excellence 2015' 'Certificate of Excellence 2017'
 'Certificate of Excellence 2013' 'Certificate of Excellence 2014'
 'Certificate of Excellence 2012' 'Certificate of Excellence 2011']

Valores únicos en 'meals': 161 valores diferentes
[nan 'Lunch' 'Lunch, Dinner'

In [12]:
## Count nulls

def null_summary(df):
    null_counts = df.isnull().sum() 
    null_percentage = (null_counts / len(df)) * 100 
    null_data = pd.DataFrame({
        'Null Count': null_counts,
        'Null Percentage': null_percentage
    })
    return null_data

# Missing data for each column individually
null_summary_df = null_summary(df_filtered)
print(null_summary_df)

# Missing data in the whole data matrix
total_missing_percentage = df_filtered.isnull().sum().sum() / (df_filtered.size) * 100
print("Total missing data (%): ", total_missing_percentage)

                             Null Count  Null Percentage
price_level                           0         0.000000
vegan_options                         0         0.000000
gluten_free                           0         0.000000
open_days_per_week                  444        11.102776
avg_rating                            0         0.000000
total_reviews_count                   0         0.000000
food                                  4         0.100025
service                               3         0.075019
atmosphere                         1157        28.932233
excellent                             0         0.000000
location                              0         0.000000
last_certificate_excellence        1119        27.981995
cuisine_nation                      630        15.753938
meals                              1686        42.160540
Total missing data (%):  9.007609045118423
